# Regresja logistyczna
Te ćwiczenie polegać będzie na stworzeniu regresji logistycznej opisującej prawdopodobieństwo przeżycia na [Titanicu](https://www.kaggle.com/competitions/titanic/data).  


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel("Dane_Titanic.xlsx")
df.head(10)

Kasujemy kolumne *Cabin*, która prawie nie miała zawartości i usuwamy braki danych

In [ ]:
df = df.drop(['Cabin'], axis=1)
df = df.dropna()

In [ ]:
df.info()

## Wybieramy zmienne objaśniane:
W pierwszej kolejności wybierzmy z danych informację czy dana osoba przeżyła (zmienna *survived*). Przypiszmy te dane do obiektu *numpy*.

Nazwijmy obiekt przez zmienną *y* - będzie się pojawiała w kolejnych blokach.

In [ ]:
df_y = df.Survived
df_y.head(10)

In [ ]:
y = df_y.to_numpy()
y

Wybierzmy teraz ramkę danych, które objaśnią nasze dane. W pierwszej kolejności użyjmy informacji wyłącznie ilościowych

1.   Wiek (zmienna *Age*)
2.   Opłata za bilet (zmienna *Fare*)

Nazwijmy obiekt przez zmienną *x* - będzie się pojawiała w kolejnych blokach.


In [ ]:
df.keys()

In [ ]:
df_x = df[["Age", "Fare"]]
df_x.head(10)

In [ ]:
x = df_x.to_numpy()
x

## Generujemy pierwszy model
Do przygotowania modelu potrzebna będzie nam biblioteka *sklearn* - importujemy z niej algorytmy odpowiadające za problemy klasyfikacji.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

Generujemy model i dopasowujemy go do danych

In [ ]:
model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(x, y)

Możemy odczytać parametry naszego modely - biblioteka *sklearn* sama w sobie ma mało rozbudowane wydruki. Pod tym względem zdecydowanie łatwieje wybierać pakiet R.

In [ ]:
model.coef_

In [ ]:
model.intercept_

Do wyświetlania wyników możemy posłużyć się biblioteką *statsmodels.api* - ma ona jednak trochę inną składnie i domyślne ustawienia modeli.

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(y,x)

result=logit_model.fit()

print(result.summary2())

## Tworzymy predykcje
Przyjemnym apsektem jest jednak łatwe tworzenie predykcji - generujemy je następująco:

In [ ]:
model.predict(x)

Możemy też przyjrzeć się estymowanemu prawdopodobieństwu każdej z klas - tu generuje je dla pierwszych 10 obserwacji

In [ ]:
model.predict_proba(x[1:10,])

Łatwo też sprawdzimy dokładność naszych oszacowań:

In [ ]:
model.score(x, y)

Oraz to w jakich okolicznościach myli się nasz model:

In [ ]:
cm = confusion_matrix(y, model.predict(x))
cm

In [ ]:
print(classification_report(y, model.predict(x)))

# Praca z danymi - rozbudowujemy model
Jeżeli oglądaliśmy film *Titanic* to pamiętamy, że Kate Winslet miała możliwość opuszczenia statku szalupą, a Di Caprio już nie. Może warto dodać zmienną dotyczącą płci do modelu.

W danych ma ona jednak charakter *string*. Przepiszmy to na liczby:

In [ ]:
df['Sex_Q'] = df.Sex.apply(lambda x: 1 if x == 'female' else 0)
df.head(10)

Zastąpmy dane dot. wieku naszą płcią:

In [ ]:
df_x = df[["Fare", "Sex_Q"]]
x = df_x.to_numpy()
x

Estymujemy na nowo model i tworzymy wydruk arkusza:

In [ ]:
model2 = LogisticRegression(solver='liblinear', random_state=0)
model2.fit(x, y)
model2.score(x, y)

In [ ]:
print(classification_report(y, model2.predict(x)))

## Testowanie wyników na danych spoza próby - Overfitting
W danych, które używaliśmy były tez osoby które nie podały wieku - znamy jednak ich płeć i klasę biletu. Zobaczmy jakie będą wyniki predykcji dla tej grupy.

W pierwszej kolejności filtrujemy tylko obserwacje bez wieku - nie weszły do naszej próby:

In [ ]:
df2 = pd.read_excel("Dane_Titanic.xlsx")
df2 = df2[df2.Age.isna()]
df2.info()

Aplikujemy przekształcenie do zmiennej dla płci:

In [ ]:
df2['Sex_Q'] = df2.Sex.apply(lambda x: 1 if x == 'female' else 0)
df2 = df2[["Survived","Fare", "Sex_Q"]].dropna()

Ponownie przekształcamy szybko nasze wartości do macierzy numpy:

In [ ]:
y2 = df2.Survived.to_numpy()
x2 = df2[["Fare", "Sex_Q"]].to_numpy()

Wyniki są trochę słabsze, ale jest nieźle:

In [ ]:
model2.score(x2, y2)

Pamiętajmy o procedurze: Jeżeli chcemy sprawdzić jakość modelu, zwykle rozdzielamy nasze dane na dwa zestawy - ten na którym uczymy algorytm, oraz ten na którym go testujemy

## Dodajemy kilkupoziomowe zmienne porządkowe.
Dodajmy do modelu zmienną opisującą miejsce w którym pasażerowie zaczęli rejs (zmienna *Embarked*). Zawiera ona 3 lokalizacje:
1. C = Cherbourg
2. Q = Queenstown
3. S = Southampton

Pandas pozwala nam szybko rozłożyć zmienną na dane binarne:


In [ ]:
dummies = pd.get_dummies(df.Embarked)
dummies.head()

Aby dołączyć zmienną do modelu usuwamy jeden z poziomów - będzie on naszym punktem odniesienia. Usuniemy kolumnę C - parametry będa mówiły, czy osoby z Queenstown i Southampton miały większe szanse na przeżycie niż osoby z Cherbourga.  

In [ ]:
z = dummies[["Q", "S"]].to_numpy()
x = np.concatenate((x,z), axis = 1)
x

Oszacujemy model z dodatkowymi zmiennymi:

In [ ]:
logit_model=sm.Logit(y,x)

result=logit_model.fit()

print(result.summary2())

**Uwaga** Więcej zastosowań metod statystycznych z Python można znaleźc w dokumentacji biblioteki [*statsmodel*](https://www.statsmodels.org/dev/gettingstarted.html)